#### Imports

In [2]:
import PyPDF2
import csv
import re
import json
import os
import us

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [3]:
year = 1998

pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 568


In [14]:
abbr_keys = ["TX", "ML", "PV", "OS", "DT",\
             "OD", "OR", "IO", "ID", "IR", "RR",\
             "AD", "DD", "HV", "PW",\
             "SS", "GH", "PH", "OH", "TC", "HH", "CM", "CH", "CO", "SG", "SC", "CJ",\
             "MC", "MD", "FG", "PI"]

#### Test on a page of data
- data starts on page 12 (getPage(11))
- split to each city, then split into data centers, clean city name and keys!
- To fix?
    - Some center names includes the capitalized city name in front
    - If the city name is of two words, only the first word would be included...

In [277]:
from difflib import SequenceMatcher

In [55]:
pagen = 13
print(pdfReader.getPage(pagen - 1).extractText())

DEMOPOLISWestAlabamaMentalHealthCenterSubstanceAbuseProgram1215WalnutAvenueSouthDcmopolis,AL36732(334)289-2410Hothncs:(800)239-2901(334)289-2410TXPVOS/ORIO/CM/MDDOTHANSpectraCare831JohnDOdomRoadDothan,AL36303(334)794-3771Hotline:(334)794-0700TX/RR/SSOHFAIRHOPEBaldwinCountyMentalHealthCenter372SouthGreenoRoadFairhope,AL36532-1905(334)990-8961Hotlines:(800)738-2871(800)558-8295TXPVOS/ORIO/SSCM/MCFLORENCERiverbcndCenterforMentalHealthSubstanceAbuseServices635WestCollegeStreetFlorence,AL35630(256)764-3431Hotline:(256)764-3431TXPV/OR/ADDDHVPW/SSPHTCCM/MDPIGADSDENCherokee/Etowah/DeKalbMentalHealthCenterSubstAbuseServs901GoodyearAvenueGadsden,AL3S903(256)492-7800Hotline:(256)492-7800TXPVOS/ORIO/DD/CM/MDTheBridgeInc3232LaySpringsRoadGadsden,AL35904(256)546-6324TXPVOS/IORR/ADDD/SSSCGUNTERSVILLEMarshall/JacksonMHAuthorityMountainLakesBehavioralHealthCare22165U.S.Highway431Guntersville,AL35976(2S6)582-4465Hotlines:(256)582-5339(256)582-4465TXPVOS/ORIORR/DD/SSHHCM/MDPIHUNTSVILLECrestwoodMedicalCtr

In [302]:
pagen = 17
print(pdfReader.getPage(pagen - 1).extractText())

METLAKATLAAnnetteIslandServiceUnitFamilyServicesMetlakatla,AK99926(907)886-4325TXPVOS/ORIO/AD/SSNOMENorthernLightsRecoveryCenter5thAvenueandDivisionStreetsCommunityHealthServicesBuildingNome,AK99762(907)443-3344TXOS/ORIORR/DDPW/SSTC/MDFGPIPETERSBURGChangingTidesCounselingServices201NorthNordicStreetSuites204and205Pctcrsburg,AK99833(907)772-3552TXPVOS/OR/AD/SS/MDSAINTPAULISLANDPribilofCounselingCenterSaintPaulIsland,AK99660(907)546-2342TXPVOS/OR/AD/SSOHSANDPOINTEasternAleutianTribesIncMainStreetSandPoint,AK99661(907)383-5421Hotline:(800)478-2673TXPVOS/ORIO/ADDD/SSCMSELDOVIASeldoviaVillageTribe/SKIAP274MainStreetSeldovia,AK99663-0197(907)234-7807TXPVOS/OR/AD/SSSEWARDSewardLifeActionCouncil504AdamsStreetSeward,AK99664(907)224-5257Hotline:(907)224-3027TXPVOS/OR/SSCM/MDPlSITKARavensWay/SEARHCAdolescentResidentialTreatmentProg222TongassDriveSitka,AK99835(907)966-8714TXOS/RR/ADDD/SS/MDFGPISitkaPreventionandTrtServicesIncHeadquarters509LincolnStreetSitka,AK99835(907)747-3500TXPVOS/ORIO/AD/SSHH

- A function to match city names

In [295]:
def fuzzy_search(search_key, text, strictness):
    '''
    Find the position of a word in a string, with some tolerance.
    *splits string by capital letter!
    '''
    for pos in re.finditer(r"[A-Z]", text):
        word_to_test = text[pos.start(): pos.start() + len(search_key)]
        similarity = SequenceMatcher(None, word_to_test.lower(), search_key.lower())
        if similarity.ratio() > strictness:
            return (pos.start(), pos.start() + len(search_key))

In [298]:
text = "WestAlabamaMentalHealthCenterSubstanceAbuseProgram1215WalnutAvenueSouthDcmopolis,AL36732(334)289-2410Hothncs:(800)239-2901(334)289-2410TXPVOS/ORIO/CM/MD"

- test

In [299]:
fuzzy_search("DEMOPOLIS", text, 0.8)

(71, 80)

In [300]:
text[71: 80]

'Dcmopolis'

- Split data into a list containing a list of facilities in that city

In [313]:
def split_cap_words(string):
    return re.sub(r"(\w)([A-Z])", r"\1 \2", string)

In [354]:
def split_data(rawdata, strictness):
    '''
    Roughly split data into each facility, returns list of list.
    '''
    data = []
    temp_city = []
    #find the city name
    m = re.search("[A-Z]+", rawdata).end()
    city = rawdata[:m - 1]
    
    
    rawdata = rawdata[m - 1:]
    a, b = fuzzy_search(city, rawdata, strictness)
    state_clean = split_cap_words(rawdata[a:b])
    temp_city.append(state_clean)
    
    
    m = re.search("[A-Z]{2}[A-Z/]", rawdata[b:]).start() #start of the keys
    j = 0
    temp_facil = [rawdata[:b+m]]
    temp_facil.append('')
    while rawdata[b+m+j:b+m+j+2] in abbr_keys:
        temp_facil[-1] += ' ' + rawdata[b+m+j:b+m+j+2]
        if rawdata[b+m+j+2] == "/":
            j += 3
        else:
            j += 2
    temp_facil[-1] = temp_facil[-1].strip()
    temp_city.append(temp_facil)
    data.append(temp_city)
    return data

In [355]:
pagen = 17
rawdata = pdfReader.getPage(pagen - 1).extractText()
rawdata

'METLAKATLAAnnetteIslandServiceUnitFamilyServicesMetlakatla,AK99926(907)886-4325TXPVOS/ORIO/AD/SSNOMENorthernLightsRecoveryCenter5thAvenueandDivisionStreetsCommunityHealthServicesBuildingNome,AK99762(907)443-3344TXOS/ORIORR/DDPW/SSTC/MDFGPIPETERSBURGChangingTidesCounselingServices201NorthNordicStreetSuites204and205Pctcrsburg,AK99833(907)772-3552TXPVOS/OR/AD/SS/MDSAINTPAULISLANDPribilofCounselingCenterSaintPaulIsland,AK99660(907)546-2342TXPVOS/OR/AD/SSOHSANDPOINTEasternAleutianTribesIncMainStreetSandPoint,AK99661(907)383-5421Hotline:(800)478-2673TXPVOS/ORIO/ADDD/SSCMSELDOVIASeldoviaVillageTribe/SKIAP274MainStreetSeldovia,AK99663-0197(907)234-7807TXPVOS/OR/AD/SSSEWARDSewardLifeActionCouncil504AdamsStreetSeward,AK99664(907)224-5257Hotline:(907)224-3027TXPVOS/OR/SSCM/MDPlSITKARavensWay/SEARHCAdolescentResidentialTreatmentProg222TongassDriveSitka,AK99835(907)966-8714TXOS/RR/ADDD/SS/MDFGPISitkaPreventionandTrtServicesIncHeadquarters509LincolnStreetSitka,AK99835(907)747-3500TXPVOS/ORIO/AD/SSH

In [356]:
split_data(rawdata, 0.8)

[['Metlakatla',
  ['AnnetteIslandServiceUnitFamilyServicesMetlakatla,AK99926(907)886-4325',
   'TX PV OS OR IO AD SS']]]

In [247]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list of two terms
    '''
    
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if span[1] != len(cn_sa):
                return ([split_cap_words(cn_sa[:span[0]]), split_cap_words(cn_sa[span[0]:])])
    return [split_cap_words(cn_sa), '']


def split_address(csp):
    '''
    Split a string containing city, state and postal code.
    Inputs:
        csp(str): a string containing city, state and postal code
    Returns: a list of three items
    '''
    #print("Split address:", csp)
    if ',' in csp:
        c, s_p = csp.split(',')
    else:
        c, s_p = csp.split('.')
    m = re.search(r'\d', s_p).span()[0]
    s = s_p[:m]
    p = s_p[m:]
    return [split_cap_words(c), s, p]

In [263]:
def clean_data(writer, cts, year, pagen):
    '''
    Write data to csv
    '''
    for ct in cts:
        ct = ct[0]
        print("\n", ct)
        data_holder = [year, pagen]
        #Center name
        m = re.search(r",", ct)
        if m == None:
            m = re.search(r"\.", ct).span()[0]
        else:
            m = m.span()[0]
        m = re.search(r"(?s:.*)[A-Z]", ct[:m]).span()[1]
        if ct[m - 1:m + 3] == "City": #if the state name is xx city
            m = re.search(r"(?s:.*)[A-Z]", ct[:m-1]).span()[1]
        data_holder += split_center_street(ct[:m - 1])
        #City, State, Postal_code
        aftercn = ct[m - 1:]
        m = re.search(r"[^,\.][A-Z]{2}[A-Z/][A-Z/105]", aftercn).span()[0] + 1 
        csp = split_address(aftercn[:m])
        p_phone = csp[2]
        if p_phone[5] == "-":
            csp[2] = p_phone[:10]
            phone = p_phone[10:]
        else:
            csp[2] = p_phone[:5]
            phone = p_phone[5:]
        csp[2] = csp[2].replace("O", "0").replace("S", "5")
        data_holder += csp
        #Phone
        data_holder.append(phone.replace("O", "0").replace("S", "5"))
        #Keys
        data_holder += [aftercn[m:].replace("0", "O").replace("5", "S").replace("1", "I")]
        writer.writerow(data_holder)
        #print(data_holder)

In [265]:
def run(year, start_page, end_page = 0):

    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1

    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])
    #scrape
    for pagen in range(start_page, end_page + 1):
        print(pagen)
        rawdata = pdfReader.getPage(pagen - 1).extractText()
        data = split_data(rawdata)
        
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            clean_data(writer, data, year, pagen)

In [266]:
run(1998, 12, 561) #12

12

 ALABAMAALEXANDERCITYLtghthouseofTallapoosaCountyInc36FranklinStreetAlexanderCity,AL35010(205)234-4894TX/RR/HHANNISTON

 AnnistonFellowshipHouseInc106East22ndStreetAnniston,AL36201(256)236-7229TX/RR/HHCJ

 Calhoun/CleburncMentalHealthCenterNewDirections331East8thStreetAnniston,AL36202(256)236-3403Hotlines:(256)236-3403(256)236-8003TXPVOS/ORIO/ADDD/SSCMBIRMINGHAM

 AlcoholismRecoveryServicesInc2701JeffersonAvenueSWBirmingham,AL35211(205)923-6552TXPVOSDT/ORIORDRR/HV/SSOH/MD

 AletheiaHouse201FinleyAvenueWestBirmingham,AL35204(205)324-6502TXOS/ORIO/SS/MD

 BirminghamHealthCareforTheHomeless71225thStreetNorthBirmingham.AL35203(205)439-7216TXPVOS/ORIO/SS/FG

 BradfordHealthServicesBirminghamRegionalOfficeJefferson631BeaconParkwayWestSuite211Birmingham,AL35209(800)293-7191TXOS/ORIO/ADDD/SS

 DepartmentofVeteransAffairsMedicalCenter171711thAvenueSouthBirmingham,AL3520S(205)939-2135TXOS/OR/DD/GH/FG

 FellowshipHouseInc162512thAvenueSouthBirmingham,AL35205(205)933-2430TXPVOS/RR/DD/HH/FG

 H


 MaverickHouse7022North48thAvenueGlendale,AZ8S301(623)931-5810TXPVOS/ORRR/DD/SSTC/MCMDPI

 ThunderbirdSamaritanBehavioralHlth5555WestThunderbirdRoadGlendale,AZ85306(602)588-4799Hotline:(602)230-4357TXOSDT/ID/GH/MCMDPIGREENVALLEY

 LaFronteraCenterBLNA1151SouthLaCanadaSuite105GreenValley,AZ85614(602)625-0876TXPVOSDT/OD10ID/DDPW/CMHOLBROOK

 CommunityCounselingCentersInc105North5thAvenueHolbrook,AZ86025(520)524-6126Hotline:(520)524-6126TXPVOSDT/ODORIO/CMKAYENTA

 KayentaOutpatientTreatmentCenterKayenta,AZ86033(520)697-5570TXPVOS/OR/ADDD/SSKEARNY

 CopperCommunitiesResourceandDevelopmentInc1116TilburyStreetKearny,AZ85237(602)363-5561TXPVOS/OR/ADDD/COKINGMAN

 MohaveMentalHealthClinicSubstanceAbuseServices1750BeverlyStreetKingman,AZ86401(520)757-8111TXOS/IO/DD/SSSGLAKEHAVASUCITY

 MohaveMentalHealthClinicSubstanceAbuseServices2187SwansonStreetLakeHavasuCity,AZ86403(520)855-7806TXPVOS/ORIO/ADDD/SSCMCHLUKEAFB

 LukeAirForceBaseSubstanceAbuseProgram52MDOS/SGOMH7219NorthLitchfieldRoadLukeAFB,


 GRADYDeptofCorrections/CumminsUnitSubstanceAbuseTreatmentProgGrady,AR71644(870)479-3311x332TXOS/RR/SSTCCJ

 DeptofCorrections/VarnerUnitSubstanceAbuseTreatmentProgGrady,AR71644(870)479-3030x462TXPVOS/RR/SSCJHARRISONOMART218

 EastRidgeAvenueHarrison,AR72601-4307(870)432-6200Hotline:(870)435-6200TXPVOS/ORIO/SSHOPE

 SouthwestArkansasCounselingandMentalHealthCenter220East20thStreetHope,AR71801(870)773-4655TXOS/OR/CMHOTSPRINGS

 QuapawHouseInc812MountPineRoadHotSprings,AR71913(501)767-4456TXOSDT/ORIORDRR/SS/PIHOTSPRINGSNATIONALPARK

 BarbsPlace276LindenAvenueHotSpringsNationalPark,AR71901-3308(501)624-6185TXPVOS/RR/PW/SS/PIJONESBORO

 CrowleysRidgeDevelopmentCouncilNortheastArkansasWomensRecovery417WestJeffersonStreetJonesboro,AR72401(870)932-0228TXPVOS/ORIORR/AD/SS

 Mid/SouthHealthSystems2920McClellanDriveJoncsboro,AR72401(870)972-4000Hotlines:(870)972-4032(800)356-3035TXPVOSDT/ODOR/ADDD/CM

 SaintBernardsBehavioralHealthSubstanceAbuseTreatmentUnit2712EastJohnsonAvenueJonesboro,AR7240

 SanDiegoFreedomRanch1777BuckmanSpringsRoadCampo,CA91906(619)478-5696TXOS/RR/SS/FGCALIFORNIA20
32

 CAMPPENDLETONConsolidatedSubstanceAbuseCounselingCenterMarineCorpsBaseBuilding16105CampPendleton,CA9205S-5016(760)725-5539TXPVOS/ORIO/SS/FG

 NavalHospitalNavalAddictionsRehab/EducDepartmentBuildingH-49CampPendleton,CA92055(760)725-1561TXPV/ORIORR/DD/SSCANOGAPARK

 CabritoFoundationCabritoHouse7552RemmctAvenueCanogaPark,CA91303(818)704-5175TX/RR/SS

 PineGroveHospital7011ShoupAvenueCanogaPark,CA91307(818)348-0500Hotline:(818)348-0500TXPVOSDT/IDIR/ADDD/PHCANYONCOUNTRY

 I-ADARP27225CampPlentyRoadSuite4CanyonCountry,CA91351-2654(661)251-7691Hotline:(800)540-4673TXPVOS/ORIO/ADDDHVPW/SSCAPISTRANOBEACH

 CommunityCounselingCenterofSanJuanCapistrano/CasaDelCerroI26882-26884AvenidaLasPalmasSouthCapistranoBeach,CA92624(949)493-7388TXOSDT/RDRR/DD/SS

 BivalleyMedicalClinicCarmichael6127FairOaksBoulevardCarmichael,CA95608(530)974-8090TXMLOSDT/ODORIO/DDHVPW/SS/MDCARPINTERIA

 SalvationArmyAdultReha


 FresnoCountyHispanicCommissiononAlcoholandDrugAbuseServicesInc1444FultonStreetFresno,CA93721(S59)268-6475TXPVOS/OR

 GenesisGroupHomeSpiritofWoman728NorthEchoAvenueFresno.CA93728(559)225-9548TXPVOS/IORR/HVPW/SSTCCO

 KaiserPcrmanentcChemicalDependencyServices4785North1stStreet2ndFloorFresno,CA93726(559)448-4765Hotline:(800)559-8460TXOSDT/ODIOIDRD/ADDDHVPW/SS

 KingofKingsMensRecoveryHome2267SouthGenevaStreetFresno,CA93706(559)266-6449Hotline:(559)266-6449TXOS/RR/DDHV/SS

 KingofKingsOutpatientServices/WestFresno2385SouthFairviewAvenueSuite17Fresno,CA93706(559)237-4116Hotline:(5S9)266-6449TXOS/ORIO/DD/SS/MD

 KingofKingsPregnantPostPartumWomensProgram1350EastAnnadaleAvenueFresno,CA93706(559)486-8200Hotline:(559)266-6449TXOS/ORIO/PW/SS

 KingofKings/AlcoholandDrugFreeResidPrcgandPostPartumVisionsPrg1530WestWhitesbridgcRoadFresno,CA93706(559)264-4663Hotline:(559)266-6449TXOS/RR/PW/SS

 NuestraCasaRecoveryHome1414WestKearneyBoulevardFresno,CA93704(559)485-0501Hotline:(559)485-0501TXPVOS/

...


 RIDGECRESTCollegeHealthIPARidgecrestUnit1400NorthNormaStreetSuite13Ridgecrest,CA93555(760)375-4756Hotline:(888)343-4756TXPVOS/IO/DDPW/SSCM/MD

 TrafficandAlcoholAwarenessSchoolofKern(TAASK)443WestChurchStreetRidgecrest,CA93555(661)832-3283TX/ORRIOVISTA

 RioVistaCare125SacramentoStreetRioVista,CA94571(707)374-5243TXPVOSDT/ODOR/AD/SSRIVERSIDE10

 AcreRanchInc6067BeachStreetRiverside,CA92509(909)681-8096Hotline:(800)266-7044TXPVOSDT/RDRR/SS

 BornFree8310BaxterWayRiverside,CA92504(909)687-9922TXPVOS/RR/PW/SSTC

 KnollwoodPsychiatricandChemicalDependencyCenter5900BrocktonAvenueRiverside,CA92506(909)275-8400TXOSDT/IOIDIR/DD/PHOHCM/MCFGPI

 MyFamilyRecoveryCenterAWomensPlace/AWP4295BrocktonAvenueRiverside,CA92501(909)341-3786TXOS/RR/DDHVPW/SSTC

 MyFamilyRecoveryforWomen7211MagnoliaAvenueRiverside,CA92504(909)683-4315TXOS/IO/DDHVPW/SS/FG

 PineRidgeTreatmentCenter5995BrocktonAvenueSuiteBRiverside,CA92506(909)784-3050TXOS/ORIO/SS

 RiversideCountySubstanceAbuseProgAlcohol1777AtlantaAvenueS


 PartnersinPreventionEducationandRecovery3274RosecransStreetSanDiego,CA92110(619)222-1714TXPVOS/OR/AD/SSOHCJ

 PathfindersofSanDiegoRecoveryHome2980CedarStreetSanDiego,CA92102(619)239-7370TXOS/RR/SSHH/FG

 RandRecoveryResources2206BalboaAvenueSanDiego,CA92109(858)274-6642Hotline:(800)375-7263TXPVOS/ORIORDRR/DD/SSHH/MCFGPI

 SanDiegoCenterforPsychotherapy600BStreetSuite1420SanDiego,CA92101(619)234-8139TXOS/OR/ADDD/SG/MCMDPI

 SanDiegoCommunityTreatmentCenter50210thAvenueSanDiego,CA92101(619)239-7181TXPVOSDT/RD/SS

 SanDiegoHealthAllianceWestOffice7020FriarsRoadSanDiego,CA92108(619)718-9890TXMLDT/ODORIO/SS/MD

 SanDiegoTreatmentServicesHomeAvenueClinic3940HomeAvenueSanDiego,CA92105(619)262-8000TXMLPVOSDT/ODOR/DD/SSGHSG/MDFG

 SanDiegoYouthandCommunityServicesTeenOptions3660FairmountAvenueSanDiego,CA92105(619)521-2250Hotline:(619)239-4688TXPVOS/ORIO/AD/SSCO

 ScrippsClinicChemicalDependencyTreatmentProgram4320LaJollaVillageDriveSuite140SanDiego,CA92122-1204(858)622-0394TXOSDT/ODOR10/DD/C


 AlertDrivingInc(ADI)AdvancedDrugDiversionInstitute3150AlmadenExpresswaySuite14SSanJose,CA95118(408)445-0491TXPVOS/OR/DDHV/SS

 AlcxianAssociatesFamilyPsychologyandCounseling3110ProvoCourtSuiteASanJose,CA95127-1034(408)272-4321TXPVOS/OR/AD/SG

 AsianAmericanRecoveryServices1370TullyRoadSuite501and502SanJose,CA95122(408)271-3900TXPVOS/ORIO/AD/SS

 AsianAmericansforCommunityInvolvement2400MoorparkAvenueSuite300SanJose,CA95128-2680(408)975-2730TXPVOS/OR/ADDD/CMSCCJ

 BennyMcKeownCenter1281FlemingAvenueSanJose,CA95127(408)259-6565TXOS/RR/DDHV/SS

 BlossomsPerinatalCenterGardnerFamily3030AlumRockAvenueSanJose,CA95127(408)254-3396TXPVOS/IO/PW/SS/FG

 CentralTreatmentandRecoveryCenter976LenzenAvenueFirstFloorSanJose,CA95126(408)299-7280Hotline:(800)488-9919TXOS/OR/DD/SS/MD

 CentralValleyMethadoneClinic2425EnborgLaneSanJose,CA95128(408)885-5400Hotline:(800)488-9919TXMLPVOSDT/ODORIO/DDHVPW/SS/MD

 CharterBehavioralHealthSystemofSanJose4S5SiliconValleyBoulevardSanJose,CA95138(408)224-2020TXPVO


 DrugAbuseAlternativesCenterRedwoodEmpireAddictionsProg(REAP)2403ProfessionalDriveSuite103SantaRosa,CA95403(707)526-2999TXMLPVOSDT/ODORIO/SS/MD

 DrugAbuseAlternativesCenterTurningPoint2403ProfessionalDriveSuite101SantaRosa,CA95403(707)544-3295x41Hotline:(707)539-8868TXOS/RR/HV/SSTC/FG

 LowerLakeTransitionalLivingCenter2403ProfessionalDriveSuite101SantaRosa,CA95403(707)526-2999TXOS/ORIO/PW/SSR

 HouseOakParkFacility5136OakParkWaySantaRosa,CA95409(707)539-2948TXOS/RR/AD/SSTC

 SantaRosaTreatmentProgram1901ClevelandAvenueSuiteBSantaRosa,CA95403(707)576-0818TXMLPVOSDT/ODORIO/SS/MD

 SonomaCntyAlc/Drug/TobaccoServsDivUnityHouse920West8thStreetSantaRosa,CA95401(707)524-7450Hotline:(707)524-7450TXOS/RR/SS

 SonomaCntyDeptofHealthServicesAlcohol/DrugandTobaccoServicesDiv2759BennettValleyRoadSantaRosa,CA95404(707)524-7450Hotline:(707)524-7450TXOSDT/ORRDRR/ADDD/SSCJ/MD

 SonomaCountyIndianHealthProjectBehavioralHealthDepartment791LombardiCourtSuite101SantaRosa,CA95407(707)544-4056Hotline:(707

ValueError: too many values to unpack (expected 2)

In [243]:
pagen = 15
rawdata = pdfReader.getPage(pagen - 1).extractText()
data = split_data(rawdata)

In [244]:
data

[['ALASKAANCHORAGEAkeelaTreatmentServices2805BeringStreetSuite4Anchorage,AK99503(907)561-5266TXPVOS/ORRR/SSTCSCCJ/MD'],
 ['AlaskaHumanServicesIncOutptAlc/SubstAbuseTreatmentProg4050LakeOtisParkwaySuite111Anchorage,AK99508(907)561-4535TXOS/ORIO/SS/PI'],
 ['AlaskaNorthAddictionsRecoveryCtr4330BragawStreetAnchorage,AK99508(907)561-5537TXPVOS/ORIORR/SS/MDFG'],
 ['PlBoothMemorialYouthandFamilyServs3600East20thAvenueAnchorage,AK99508(907)279-0522TXPVOS/ORIO/ADDDPW/SS/MD'],
 ['CharterNorthBehavioralHealthSystem2530DebarrRoadAnchorage,AK99508(907)258-7575Hotlines:(907)258-7575(800)478-7575TXPVOSDT/IDIR/AD/PH'],
 ['GenesisHouseInc2825West42ndPlaceAnchorage,AK99517(907)243-5130TXPVOSDT/ORIORDRR/DD/SSCM/FG'],
 ['PlNarcoticDrugTreatmentCenterIncCenterforDrugProblems520East4thAvenueSuite102Anchorage,AK99S01-2624(907)276-6430TXMLPVOSDT/ODOR/HV/SS/MD'],
 ['PlPacificRimCounselingInc4141BStreetSuite210Anchorage,AK99503(907)561-5252TXPVOS/ORIO/ADDD/SSSGCJ/PI'],
 ['ProvidenceAlaskaMedicalCenterProvidence

In [235]:
data = [['MaxwellAirForceBaseSubstanceAbuseProgram42MDOS/SGOMH330KirkpatrickAvenueEMaxwellAFB,AL36113-6334(334)493-4534TXPVOSDT/ODORIO/PW/SSGHCM/FGMOBILE'],
 ['BradfordHealthServicesMobileOutreachMobile1000HillcrestRoadSuite304Mobile,AL36695(800)333-0906Hotlines:(334)633-0900(800)333-0906TXPVOS/ORIO/SS'],
 ['DauphinWayLodge1009DauphinStreetMobile,AL36604(256)438-4729TXOS/IORR/SSHH/FGECD'],
 ['Program29SOSpringhillAvenueMobile,AL36607(334)476-1677TX/IO/SS'],
 ['Oasis4211GovernmentBoulevardMobile,AL36693(334)666-2569TXOS/OR10/SSOH/PIMONTGOMERY'],
 ['BradfordHealthServicesMontgomeryOutreachMontgomery100MendelParkwayMontgomery,AL36117(800)873-2887Hotline:(334)244-0702TXPVOS/ORIO/AD/SS'],
 ['ChemicalAddictionsProgramInc1153AirBaseBoulevardMontgomery,AL36108(334)265-4544Hotline:(800)762-3790TXOS/IORR/AD/SS/MDFGALABAMA']]

In [236]:
save_as_filename = '{0}/{0}.csv'.format(year)

with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data, year, pagen)


 MaxwellAirForceBaseSubstanceAbuseProgram42MDOS/SGOMH330KirkpatrickAvenueEMaxwellAFB,AL36113-6334(334)493-4534TXPVOSDT/ODORIO/PW/SSGHCM/FGMOBILE

 BradfordHealthServicesMobileOutreachMobile1000HillcrestRoadSuite304Mobile,AL36695(800)333-0906Hotlines:(334)633-0900(800)333-0906TXPVOS/ORIO/SS

 DauphinWayLodge1009DauphinStreetMobile,AL36604(256)438-4729TXOS/IORR/SSHH/FGECD

 Program29SOSpringhillAvenueMobile,AL36607(334)476-1677TX/IO/SS

 Oasis4211GovernmentBoulevardMobile,AL36693(334)666-2569TXOS/OR10/SSOH/PIMONTGOMERY

 BradfordHealthServicesMontgomeryOutreachMontgomery100MendelParkwayMontgomery,AL36117(800)873-2887Hotline:(334)244-0702TXPVOS/ORIO/AD/SS

 ChemicalAddictionsProgramInc1153AirBaseBoulevardMontgomery,AL36108(334)265-4544Hotline:(800)762-3790TXOS/IORR/AD/SS/MDFGALABAMA


- Clean data a bit more from pandas dataframe
    - Name: may contain key from last term, "of" connected to previous word
    - Address: numbers a little messed up ("182217th Street Highway25" - "1822 17th Street Highway 25"), sometimes suite/p.o box included, might contain first word of the state name
    - Keys: might contain state name from the previous address or name of the next clinic, delete/move as soon as the word is not part of abbr_keys
    - Check each term that has something blanck